In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import evosax

import json


In [3]:
import xylo
import xylo.types as t
import xylo.woods
import xylo.cut

import xylo.opt

import xylo.tuning
import xylo.database

In [4]:
tuning = xylo.tuning.yamaha_YX500R
wood = t.Wood.make_E_nu(E = 29.100e9, nu = 8.20, rho = 1059)
init_mean = None

In [5]:
options = xylo.opt.Options(num_generations = 100, strategy = evosax.OpenES(popsize = 2000, num_dims = 3), absolute_tolerance = 1e-3)

In [21]:
best_splines = {}

for num_dims in range(3,10):
  options = xylo.opt.Options(num_generations = 10, strategy = evosax.OpenES(popsize = 2000, num_dims = num_dims), absolute_tolerance = 1e-3)
  init_mean = None

  for note in range(57, 89):
  # for note in range(77, 89):
    # options_ = options._replace(num_generations = gens)
    if note == 69 or note == 79:
      init_mean = None
    options_ = options._replace(init_mean = init_mean)
    # options_ = options

    bar = t.BarProps(width = 0.040, depth = 0.019, length = tuning.note_to_length(note), elements = 300, min_depth = 0.010)
    fundamental = tuning.note_to_freq(note)
    sol = xylo.opt.optimize_geometry(bar, wood, options_, fundamental)
    spline = sol.best_member

    init_mean = sol.best_member

    sections = xylo.cut.spline(bar, spline)
    swp = xylo.sweep.sweep(wood, bar, sections, t.sweep_default)

    print(note)
    print("---------------")
    print(swp.harmonics / fundamental)
    print("")

    notes = best_splines.setdefault(num_dims, {})
    notes[note] = spline

    with open(f'data/spline/{note}-s{options.strategy.num_dims}.json', 'w') as f:
      json.dump(spline.tolist(), f)
    with open(f'data/spline/{note}-s{options.strategy.num_dims}-harmonics.json', 'w') as f:
      json.dump((swp.harmonics / fundamental).tolist(), f)


iteration 0
[0.56966263 0.20591816 0.73850422] 0.15461548
[ 617.32185562 2265.04110156 4538.37488507] [0.87983801 3.22824996 6.46831906] 0.31
57
---------------
[1.00106069 2.99708737 5.9912175 ]

iteration 0
[0.59507523 0.45561532 0.75668448] 0.0017495675
[ 742.38132154 2233.4526221  4458.08628204] [0.99869361 3.00456759 5.99727141] 0.305
iteration 1: reached fitness 0.0006655799224972725
58
---------------
[1.00033039 2.99777281 5.99606113]

iteration 0
[0.62463185 0.48950457 0.72194696] 0.002487998
[ 787.47079526 2375.7290742  4702.43168911] [0.99989376 3.01659018 5.97092885] 0.3
iteration 1: reached fitness 0.0006357280653901398
59
---------------
[0.99968401 3.00000554 5.98546031]

iteration 0
[0.62836605 0.52758373 0.66480441] 0.0022796527
[ 834.90630506 2514.49162151 5013.27983631] [1.00062491 3.01358719 6.00835404] 0.295
iteration 4: reached fitness 0.00028092521824873984
60
---------------
[0.99985336 3.00018218 6.00599599]

iteration 0
[0.63806145 0.5638943  0.61357723] 0.003

KeyboardInterrupt: 

In [49]:
note = 88
# options_ = options._replace(num_generations = gens)
options_ = options._replace(init_mean = init_mean)

bar = t.BarProps(width = 0.040, depth = 0.019, length = tuning.note_to_length(note), elements = 300, min_depth = 0.010)
fundamental = tuning.note_to_freq(note)
sol = xylo.opt.optimize_geometry(bar, wood, options_, fundamental)
spline = sol.best_member

sections = xylo.cut.spline(bar, spline)
swp = xylo.sweep.sweep(wood, bar, sections, t.sweep_default)

print(note)
print("---------------")
print(swp.harmonics / fundamental)
print("")


iteration 0: reached fitness 0.0018434704979881644
88
---------------
[0.99899059 3.00122176 3.19375326]



In [21]:
xylo.database.with_db(lambda db: print(db.set_wood(57, [0.5, 0.5, 0.5])))
xylo.database.with_db(lambda db: print(db.get_wood(57)))

None
Wood(rho=501.4283176945153, E=12500000000.0, G=7500000000.0, nu=-0.16666666666666663)
